<a id='top'></a>


# Unrescaled and Barred Variables in [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), (see also [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632))
$$\label{top}$$

**Author: Zachariah Etienne**

<font color='green'>**This module has been verified against a trusted version of the code.**</font>

## A Note on Notation

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.

As a corollary, any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one: A Latin index in a four-vector will be incremented and a Greek index in a three-vector will be decremented (however, the latter case does not occur in this tutorial module).


## Introduction, Table of Contents

This module documents and constructs a number of quantities useful for building symbolic (SymPy) expressions needed in terms of unrescaled BSSN quantities (i.e., $\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}$), including:

1. [Steps 1 & 2](#initializeNRPy): Initialize core NRPy+ modules
1. [Step 3](#BSSN_barred_metric_derivs_and_extrinsic_curvature): BSSN_barred_metric_derivs_and_extrinsic_curvature()
    1. [Step 3.a](#gammabar): Barred 3-metric $\bar{\gamma}_{ij}$, $\bar{\gamma}^{ij}$
    1. [Step 3.b](#gammabar_derivs): Spatial derivatives of $\bar{\gamma}_{ij}$, including $\bar{\Gamma}^{i}_{jk}$
    1. [Step 3.c](#Abar): Traceless, conformal extrinsic curvature $\bar{A}_{ij}$, and $\bar{A}^{ij}$
1. Barred Ricci tensor $\bar{R}_{ij}$, hatted covariant derivative of barred 3-metric $\hat{D}_k \bar{\gamma}_{ij}$, $\Delta^i_{jk} = \bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}$, and $\Delta^i = \bar{\gamma}^{jk} \Delta^i_{jk}$:
    1. [RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()](#RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU)
1. Unrescaled shift vector $\beta^i$ and spatial derivatives $\beta^i_{,j}$ and $\beta^i_{,jk}$.
    1. [betaUbar_and_derivs()](#betaUbar_and_derivs)
1. Standard BSSN conformal factor $\phi$, and its derivatives $\phi_{,i}$, $\phi_{,ij}$, $\bar{D}_j \phi_i$, and $\bar{D}_j\bar{D}_k \phi_i$
    1. [phi_and_derivs()](#phi_and_derivs)
1. Code Validation
    1. [Validate results from this tutorial module against BSSN.BSSN_unrescaled_and_barred_vars NRPy+ module](#validation)

<a id='initializeNRPy'></a>

## Steps 1 & 2: Initialize core NRPy+ modules, and set spatial dimension DIM=3 \[Back to [top](#top)\]

$$\label{initializeNRPy}$$

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import sympy as sp
import indexedexp as ixp
import grid as gri
import reference_metric as rfm
# By default the reference metric is set to Spherical, and
#    the following function call sets up the reference metric
#    and related quantities, including rescaling matrices ReDD,
#    ReU, and hatted quantities.
rfm.reference_metric()

# Then we set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")

# BSSN.BSSN_rescaled_vars contains functions that declare rescaled
#      BSSN variables and registers them as gridfunctions
#      hDD, aDD, cf (conformal factor), trK, lambdaU, alpha, vetU, and betU
import BSSN.BSSN_rescaled_vars as Brv

# Step 2: Set spatial dimension (must be 3 for BSSN, as BSSN is 
#          a 3+1-dimensional decomposition of the general 
#          relativistic field equations)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

<a id='BSSN_barred_metric_derivs_and_extrinsic_curvature'></a>

## Step 3: Barred 3-metric $\bar{\gamma}_{ij}$, $\bar{\gamma}^{ij}$, needed spatial derivatives of $\bar{\gamma}_{ij}$ including $\bar{\Gamma}^{i}_{jk}$, and extrinsic curvature $\bar{A}_{ij}$, and $\bar{A}^{ij}$ \[Back to [top](#top)\]
$$\label{BSSN_barred_metric_derivs_and_extrinsic_curvature}$$

<a id='gammabar'></a>

### Step 3.a: The unrescaled, conformal 3-metric $\bar{\gamma}_{ij}$ and its inverse, $\bar{\gamma}^{ij}$ \[Back to [top](#top)\]
$$\label{gammabar}$$

Equations 1 and 34 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/pdf/1712.07658.pdf) give the relation between $\bar{\gamma}_{ij}$ and the BSSN rescaled quantity $h_{ij}$:

\begin{align}
\bar{\gamma}_{ij} &= \hat{\gamma}_{ij} + \varepsilon_{ij},\\
h_{ij} &= \varepsilon_{ij} / \text{ReDD[i][j]},
\end{align}

where as described in the [BSSN in Curvilinear Coordinates NRPy+ tutorial module](Tutorial-BSSNCurvilinear.ipynb), $\text{ReDD[i][j]}$ is the rescaling matrix constructed from the scale factors of the reference metric $\hat{\gamma}_{ij}$ as

$$
\text{ReDD[i][j]} = \text{scalefactor[i] scalefactor[j]}.
$$

Once $\bar{\gamma}_{ij}$ is computed from $h_{ij}$, the rescaling matrix $\text{ReDD[i][j]}$, and the reference metric $\hat{\gamma}_{ij}$, we can compute $\bar{\gamma}^{ij}$ via a simple $3\times 3$ symmetric matrix inversion:

In [2]:
# Step 1: All barred quantities are defined in terms of BSSN rescaled gridfunctions,
#         which we declare here in case they haven't yet been declared elsewhere.
Brv.declare_BSSN_rescaled_gridfunctions_if_not_declared_already()
hDD = Brv.hDD
aDD = Brv.aDD

# Step 2: Set spatial dimension (must be 3 for BSSN, a 3+1 decomposition of Einstein's equations of general relativity)
DIM = 3

# Step 3: gammabarDD and gammabarUU:
gammabarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        gammabarDD[i][j] = hDD[i][j]*rfm.ReDD[i][j] + rfm.ghatDD[i][j]
gammabarUU, dummydet = ixp.symm_matrix_inverter3x3(gammabarDD)

<a id='gammabar_derivs'></a>

### Step 3.b: Derivatives of the unrescaled, conformal 3-metric $\bar{\gamma}_{ij,k}$ and $\bar{\gamma}_{ij,kl}$, and associated "barred" Christoffel symbols $\bar{\Gamma}^{i}_{jk}$ \[Back to [top](#top)\]
$$\label{gammabar_derivs}$$

In the BSSN-in-curvilinear coordinates formulation, all quantities must be defined in terms of rescaled quantities $h_{ij}$ and their derivatives (evaluated using finite differences), as well as reference-metric quantities and their derivatives (evaluated exactly using SymPy). 

For example, $\bar{\gamma}_{ij,k}$ is given by:
\begin{align}
\bar{\gamma}_{ij,k} &= \partial_k \bar{\gamma}_{ij} \\
&= \partial_k \left(h_{ij} \text{ReDD[i][j]} + \hat{\gamma}_{ij}\right) \\
&= h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k},
\end{align}
where $\text{ReDDdD[i][j][k]}$ is computed within rfm.reference_metric().

In [3]:
# Step 3b.i: gammabarDDdD[i][j][k]
#               = h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k}.
gammabarDD_dD = ixp.zerorank3()
hDD_dD = ixp.declarerank3("hDD_dD","sym01")
hDD_dupD = ixp.declarerank3("hDD_dupD","sym01")
gammabarDD_dupD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            gammabarDD_dD[i][j][k] = hDD_dD[i][j][k]*rfm.ReDD[i][j] + hDD[i][j]*rfm.ReDDdD[i][j][k] \
                                   + rfm.ghatDDdD[i][j][k]
            # Compute associated upwinded derivative, needed for the \bar{\gamma}_{ij} RHS
            gammabarDD_dupD[i][j][k] = hDD_dupD[i][j][k]*rfm.ReDD[i][j] + hDD[i][j]*rfm.ReDDdD[i][j][k] \
                                     + rfm.ghatDDdD[i][j][k]

By extension, the second derivative $\bar{\gamma}_{ij,kl}$ is given by
\begin{align}
\bar{\gamma}_{ij,kl} &= \partial_l \left(h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k} \right)\\
&= h_{ij,kl} \text{ReDD[i][j]} + h_{ij,k} \text{ReDDdD[i][j][l]} + h_{ij,l} \text{ReDDdD[i][j][k]} + h_{ij} \text{ReDDdDD[i][j][k][l]} + \hat{\gamma}_{ij,kl}
\end{align}

In [4]:
# Step 3b.ii: Compute gammabarDD_dDD in terms of the rescaled BSSN quantity hDD 
#      and its derivatives, as well as the reference metric and rescaling
#      matrix, and its derivatives (expression given below):
hDD_dDD = ixp.declarerank4("hDD_dDD","sym01_sym23")
gammabarDD_dDD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                # gammabar_{ij,kl} = h_{ij,kl} ReDD[i][j]
                #                  + h_{ij,k} ReDDdD[i][j][l] + h_{ij,l} ReDDdD[i][j][k]
                #                  + h_{ij} ReDDdDD[i][j][k][l] 
                #                  + gammahat_{ij,kl}:
                gammabarDD_dDD[i][j][k][l]  = hDD_dDD[i][j][k][l]*rfm.ReDD[i][j]
                gammabarDD_dDD[i][j][k][l] += hDD_dD[i][j][k]*rfm.ReDDdD[i][j][l] + \
                                              hDD_dD[i][j][l]*rfm.ReDDdD[i][j][k]
                gammabarDD_dDD[i][j][k][l] += hDD[i][j]*rfm.ReDDdDD[i][j][k][l]
                gammabarDD_dDD[i][j][k][l] += rfm.ghatDDdDD[i][j][k][l]

Finally, we compute the Christoffel symbol associated with the barred 3-metric: $\bar{\Gamma}^{i}_{kl}$:
$$
\bar{\Gamma}^{i}_{kl} = \frac{1}{2} \bar{\gamma}^{im} \left(\bar{\gamma}_{mk,l} + \bar{\gamma}_{ml,k} - \bar{\gamma}_{kl,m} \right)
$$

In [5]:
# Step 3.b.iii: Define barred Christoffel symbol \bar{\Gamma}^{i}_{kl} = GammabarUDD[i][k][l] (see expression below)
GammabarUDD = ixp.zerorank3()
for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                # Gammabar^i_{kl} = 1/2 * gammabar^{im} ( gammabar_{mk,l} + gammabar_{ml,k} - gammabar_{kl,m}):
                GammabarUDD[i][k][l] += sp.Rational(1,2)*gammabarUU[i][m]* \
                                        (gammabarDD_dD[m][k][l] + gammabarDD_dD[m][l][k] - gammabarDD_dD[k][l][m])

<a id='Abar'></a>

### Step 3.c: Trace-free conformal extrinsic curvature $\bar{A}_{ij}$ and its contravariant version $\bar{A}^{ij}$ \[Back to [top](#top)\]

$$\label{Abar}$$

The trace-free conformal extrinsic curvature $\bar{A}_{ij}$ is related to the BSSN rescaled quantity $a_{ij}$ and the rescaling matrix via:
$$
\bar{A}_{ij} = \text{ReDD[i][j]} a_{ij},
$$
and the contravariant version $\bar{A}^{ij}$ is simply given by application of the raising operators (a.k.a., the inverse 3-metric) $\bar{\gamma}^{jk}$:
$$
\bar{A}^{ij} = \bar{\gamma}^{ik}\bar{\gamma}^{jl} \bar{A}_{kl}.
$$

In [6]:
# Step 3.c: Compute AbarUU and AbarDD, in terms of rescaled BSSN quantities a_{ij},
#          rescaling matrix ReDD[i][j], and gammabar^{ij}
AbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        # Abar_{ij} = a_{ij} ReDD[i][j]
        AbarDD[i][j] = aDD[i][j] * rfm.ReDD[i][j]

AbarUU = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                # Abar^{ij} = gammabar^{ik} gammabar^{jl} Abar_{kl}
                AbarUU[i][j] += gammabarUU[i][k]*gammabarUU[j][l]*AbarDD[k][l]

### Computing $\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j}$

First note that the covariant derivative of a metric with respect to itself is zero
$$\hat{D}_{l} \hat{\gamma}_{ij} = 0,$$
so 
$$\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} = \hat{D}_{k} \hat{D}_{l} \left(\hat{\gamma}_{i j} + \varepsilon_{ij}\right) = \hat{D}_{k} \hat{D}_{l} \varepsilon_{ij}.$$

Next, the covariant derivative of a tensor is given by (from the [wikipedia article on covariant differentiation](https://en.wikipedia.org/wiki/Covariant_derivative)):
\begin{align}
  {(\nabla_{e_c} T)^{a_1 \ldots a_r}}_{b_1 \ldots b_s} = {}
    &\frac{\partial}{\partial x^c}{T^{a_1 \ldots a_r}}_{b_1 \ldots b_s} \\
    &+ \,{\Gamma ^{a_1}}_{dc} {T^{d a_2 \ldots a_r}}_{b_1 \ldots b_s} + \cdots + {\Gamma^{a_r}}_{dc} {T^{a_1 \ldots a_{r-1}d}}_{b_1 \ldots b_s} \\
    &-\,{\Gamma^d}_{b_1 c} {T^{a_1 \ldots a_r}}_{d b_2 \ldots b_s} - \cdots - {\Gamma^d}_{b_s c} {T^{a_1 \ldots a_r}}_{b_1 \ldots b_{s-1} d}.
\end{align}

Therefore, 
$$\hat{D}_{l} \bar{\gamma}_{i j} = \hat{D}_{l} \varepsilon_{i j} = \varepsilon_{i j,l} - \hat{\Gamma}^m_{i l} \varepsilon_{m j} -\hat{\Gamma}^m_{j l} \varepsilon_{i m}.$$

Since the covariant first derivative is a tensor, the covariant second derivative is given by (same as [Eq. 27 in Baumgarte et al (2012)](https://arxiv.org/pdf/1211.6632.pdf))
\begin{align}
\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} &= \hat{D}_{k} \hat{D}_{l} \varepsilon_{i j}  \\
&= \partial_k \hat{D}_{l} \varepsilon_{i j}
 - \hat{\Gamma}^m_{lk} \left(\hat{D}_{m} \varepsilon_{i j}\right) 
 - \hat{\Gamma}^m_{ik} \left(\hat{D}_{l} \varepsilon_{m j}\right)
 - \hat{\Gamma}^m_{jk} \left(\hat{D}_{l} \varepsilon_{i m}\right),
\end{align}

where the first term is the partial derivative of the expression already derived for $\hat{D}_{l} \varepsilon_{i j}$:

\begin{align}
\partial_k \hat{D}_{l} \varepsilon_{i j} &= \partial_k \left(\varepsilon_{ij,l} - \hat{\Gamma}^m_{i l} \varepsilon_{m j} -\hat{\Gamma}^m_{j l} \varepsilon_{i m} \right) \\
&= \varepsilon_{ij,lk} - \hat{\Gamma}^m_{i l,k} \varepsilon_{m j} - \hat{\Gamma}^m_{i l} \varepsilon_{m j,k} - \hat{\Gamma}^m_{j l,k} \varepsilon_{i m} - \hat{\Gamma}^m_{j l} \varepsilon_{i m,k}.
\end{align}

In terms of the evolved quantity $h_{ij}$, the derivatives of $\varepsilon_{ij}$ are given by:
\begin{align}
\varepsilon_{ij,k} &= \partial_k \left(h_{ij} \text{ReDD[i][j]}\right) \\
&= h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]},
\end{align}
and
\begin{align}
\varepsilon_{ij,kl} &= \partial_l \left(h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} \right)\\
&= h_{ij,kl} \text{ReDD[i][j]} + h_{ij,k} \text{ReDDdD[i][j][l]} + h_{ij,l} \text{ReDDdD[i][j][k]} + h_{ij} \text{ReDDdDD[i][j][k][l]}.
\end{align}

Before returning to NRPy+, let's first define some NRPy+ notation:
* $\hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{ij;\hat{l}} =$ gammabarDD_DhatD[i][j][l]
* $\partial_k \hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{ij;\hat{l},k} =$ gammabarDD_DhatD_dD[i][j][l][k]
* $\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{ij;\hat{l}\hat{k}} =$ gammabarDD_DhatDD[i][j][l][k]

In [7]:
if par.parval_from_str("BSSN.BSSN_rescaled_vars::detgbarOverdetghat_equals_one") == "False":
    detgbarOverdetghat = gri.register_gridfunctions("AUX", ["detgbarOverdetghat"])
    detgbarOverdetghatInitial = gri.register_gridfunctions("AUX", ["detgbarOverdetghatInitial"])
    print("Error: detgbarOverdetghat_equals_one=\"False\" is not fully implemented yet.")
    exit(1)
else:
    detgbarOverdetghat = sp.sympify(1)

# Step 8d: Define detgammabar, detgammabar_dD, and detgammabar_dDD (needed for \partial_t \bar{\Lambda}^i below)
detgammabar = detgbarOverdetghat * rfm.detgammahat
detgammabar_dD = ixp.zerorank1()
if par.parval_from_str("BSSN.BSSN_rescaled_vars::detgbarOverdetghat_equals_one") == "False":
    detgbarOverdetghat_dD = ixp.declarerank1("detgbarOverdetghat_dD")
else:
    detgbarOverdetghat_dD = ixp.zerorank1()
for i in range(DIM):
    detgammabar_dD[i] = detgbarOverdetghat_dD[i] * rfm.detgammahat + detgbarOverdetghat * rfm.detgammahatdD[i]

detgammabar_dDD = ixp.zerorank2()
if par.parval_from_str("BSSN.BSSN_rescaled_vars::detgbarOverdetghat_equals_one") == "False":
    detgbarOverdetghat_dDD = ixp.declarerank2("detgbarOverdetghat_dDD", "sym01")
else:
    detgbarOverdetghat_dDD = ixp.zerorank2()

for i in range(DIM):
    for j in range(DIM):
        detgammabar_dDD[i][j] = detgbarOverdetghat_dDD[i][j] * rfm.detgammahat + \
                                detgbarOverdetghat_dD[i] * rfm.detgammahatdD[j] + \
                                detgbarOverdetghat_dD[j] * rfm.detgammahatdD[i] + \
                                detgbarOverdetghat * rfm.detgammahatdDD[i][j]

In [8]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-BSSN_unrescaled_and_barred_vars.ipynb
!pdflatex -interaction=batchmode Tutorial-BSSN_unrescaled_and_barred_vars.tex
!pdflatex -interaction=batchmode Tutorial-BSSN_unrescaled_and_barred_vars.tex
!pdflatex -interaction=batchmode Tutorial-BSSN_unrescaled_and_barred_vars.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-BSSN_unrescaled_and_barred_vars.ipynb to latex
[NbConvertApp] Writing 49090 bytes to Tutorial-BSSN_unrescaled_and_barred_vars.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
